## Name & ID

In [1]:
student_name = "Amin Fathi"
student_id =  400722102

## Import libraries 

In [2]:
import sys, re, math, random, os, string

In [3]:
!gdown --id 1vGaCn3uEV2hdCdmmq8eZK3WCGKC94aYV
!gdown --id 1OR92iOvaa3sDUfpw8HSYnzZ1eYC8tF3-
!gdown --id 1klJq7AAvgW5KTOCJa5fiXT2gJGIZFIPo

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1vGaCn3uEV2hdCdmmq8eZK3WCGKC94aYV
To: /content/Shahnameh.txt
100% 2.38M/2.38M [00:00<00:00, 104MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OR92iOvaa3sDUfpw8HSYnzZ1eYC8tF3-
To: /content/Zahhak.txt
100% 8.35k/8.35k [00:00<00:00, 6.70MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: htt

# Implementation section

## part a

In [4]:
def tokenize(text):
    return re.findall(r"[\w]+|[" + string.punctuation + "]", text)

## part b

In [5]:
def ngrams(n, tokens):
    for i in range(n):
        tokens.insert(0, "<START>")
    tokens.append("<END>")
    ngrams = []
    for i in range(n, len(tokens)):
        ngrams.append((tuple(tokens[i - n + 1 : i]), tokens[i]))
    return ngrams

## NgramModel class

In [6]:
class NgramModel(object):

    def __init__(self, n):
        self.n = n
        self.grams = []

    def update(self, sentence):
        tokens = tokenize(sentence)
        self.grams += ngrams(self.n, tokens)

    def prob(self, context, token):
        ctx_count = 0
        match_count = 0
        for i, v in enumerate(self.grams):
            if v[0] == context:
                ctx_count = ctx_count + 1
                if v[1] == str(token):
                    match_count = match_count + 1
        # protect against divide by 0 case
        if ctx_count == 0:
            return 0
        return float(match_count) / float(ctx_count)

    def random_token(self, context):
        T = [
            self.grams[i][1]
            for i in range(len(self.grams))
            if self.grams[i][0] == context
        ]
        T.sort()
        r = random.random()
        # index = int(math.ceil(r*(len(T)-1)))
        if len(T) != 0:
            index = int(r * len(T))
            if index > len(T) - 1:
                index = len(T) - 1
            return T[index]
        else:
            return " "

    def random_text(self, token_count):
        context = []
        for i in range(self.n - 1):
            context.append("<START>")
        tokens = []
        for i in range(token_count):
            toke = self.random_token(tuple(context))
            tokens.append(toke)
            if self.n > 1:
                if toke != "<END>":
                    context.append(toke)
                    context = context[1:]
                else:
                    context = []
                    for x in range(self.n - 1):
                        context.append("<START>")
        sentence = " ".join(word for word in tokens)
        return sentence

    def perplexity(self, sentence):
        perplexity = 0.0
        tokens = tokenize(sentence)
        test_ngrams = ngrams(self.n, tokens)
        for i in range(len(test_ngrams)):
            perplexity += math.log(
                1.0 / self.prob(test_ngrams[i][0], test_ngrams[i][1])
            )
        perplexity = math.exp(perplexity)
        return perplexity ** (1.0 / len(test_ngrams))



## part f

In [7]:
def create_ngram_model(n, path):
    f = open(path, "r")
    doc = NgramModel(n)
    for line in f:
        doc.update(line)
    return doc

# Check section

## part a

In [8]:
tokenize("منم کاوه دادخواه یکی بی‌زیان مرد آهنگرم")

['منم', 'کاوه', 'دادخواه', 'یکی', 'بی', 'زیان', 'مرد', 'آهنگرم']

In [9]:
tokenize("I’m Kaveh, and a blacksmith, sire")

['I', 'm', 'Kaveh', ',', 'and', 'a', 'blacksmith', ',', 'sire']

## part b

In [10]:
ngrams(1, ["کاوه", "اهنگر", "دادخواه"])

[((), 'کاوه'), ((), 'اهنگر'), ((), 'دادخواه'), ((), '<END>')]

In [11]:
ngrams(2, ["کاوه", "اهنگر", "دادخواه"])

[(('<START>',), 'کاوه'),
 (('کاوه',), 'اهنگر'),
 (('اهنگر',), 'دادخواه'),
 (('دادخواه',), '<END>')]

In [12]:
ngrams(3, ["کاوه", "اهنگر", "دادخواه"])

[(('<START>', '<START>'), 'کاوه'),
 (('<START>', 'کاوه'), 'اهنگر'),
 (('کاوه', 'اهنگر'), 'دادخواه'),
 (('اهنگر', 'دادخواه'), '<END>')]

## part c

In [13]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
print(m.prob((), "کاوه"))
print(m.prob((), "ضحاک"))
print(m.prob((), "<END>"))

0.3
0.1
0.2


In [14]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
print(m.prob(("<START>",), "کاوه"))
print(m.prob(("آهنگر",), "ضحاک"))
print(m.prob(("کاوه",), "ماردوش"))

1.0
0.3333333333333333
0.0


## part d

In [15]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(1)
[m.random_token(()) for i in range(8)]

['<END>', 'کاوه', 'کاوه', 'آهنگر', 'آهنگر', 'آهنگر', 'ضحاک', 'کاوه']

In [16]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(2)
print([m.random_token(("<START>",)) for i in range(6)])
print([m.random_token(("آهنگر",)) for i in range(6)])

['کاوه', 'کاوه', 'کاوه', 'کاوه', 'کاوه', 'کاوه']
['کاوه', '<END>', 'ضحاک', 'ضحاک', 'ضحاک', '<END>']


## part e

In [17]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(1)
m.random_text(16)

'<END> کاوه کاوه آهنگر آهنگر آهنگر ضحاک کاوه <END> <END> کاوه آهنگر کاوه <END> آهنگر کاوه'

In [18]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
random.seed(2)
m.random_text(16)

'کاوه آهنگر <END> کاوه آهنگر کاوه آهنگر <END> کاوه آهنگر ضحاک ستمگر <END> کاوه آهنگر کاوه'

## part f

In [19]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Shahnameh.txt") 
m.random_text(16)

'when him don These , , fame and be to he his King , and <END>'

In [20]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Zahhak.txt") 
m.random_text(16)

'خروشید یکی شاه <END> چو هر مغز کاوه نبرد نیکی بباید گونه گرد <END> ناشکبیم لرزان'

In [21]:
# No random seeds, so your results may vary
m = create_ngram_model(1, "Akhavan.txt") 
m.random_text(16)

'حمله دو خون کلبه دایم و بتوان پایش چه حمله رفتن کلبه ما خورده آلوده باد'

In [22]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Shahnameh.txt") 
m.random_text(16)

'it . Tus . Virtue is worthy of is my murdered by day <END> bad that'

In [23]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Zahhak.txt") 
m.random_text(16)

'به خوبی بجستند پیوند او <END> سراندر کشید و خرد <END> پسند آمدش کان سخن جز'

In [24]:
# No random seeds, so your results may vary
m = create_ngram_model(2, "Akhavan.txt") 
m.random_text(16)

'خروشد باد باران است <END> ز ابری ساکت و کولاک رعب انگیز و بر پرده های'

In [25]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Shahnameh.txt") 
m.random_text(16)

'<END> in his <END> you ; <END> PIRUZ : the area around Amol and marched toward'

In [26]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Zahhak.txt") 
m.random_text(16)

'چو شد ساخته کار گرز گران <END> <END> <END> <END> پسند آمدش کار پولادگر <END> <END>'

In [27]:
# No random seeds, so your results may vary
m = create_ngram_model(3, "Akhavan.txt") 
m.random_text(16)

'که این خون خون فرزندان صحراست <END> <END> <END> بلای نیستی سرمای پر سوز <END> گر'

In [28]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Shahnameh.txt") 
m.random_text(16)

'And let my heart shun sin , so that not one of the <END> lovely as'

In [29]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Zahhak.txt") 
m.random_text(16)

'<END> <END> <END> <END> بباید بدین بود همداستان <END> کسی کاو هوای فریدون کند <END> <END>'

In [30]:
# No random seeds, so your results may vary
m = create_ngram_model(4, "Akhavan.txt") 
m.random_text(16)

'زمین سرد است و برف آلوده و تر <END> دو دشمن در کمین ماست دایم <END>'

## part g

In [31]:
m = NgramModel(1)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
m.perplexity("کاوه آهنگر")

3.815714141844439

In [32]:
m = NgramModel(2)
m.update("کاوه آهنگر ضحاک ستمگر")
m.update("کاوه آهنگر کاوه آهنگر")
m.perplexity("کاوه آهنگر")

1.4422495703074085